In [2]:
from google.colab import files

uploaded = files.upload()


Saving api_data_aadhar_enrolment_1000000_1006029.csv to api_data_aadhar_enrolment_1000000_1006029.csv
Saving api_data_aadhar_enrolment_500000_1000000.csv to api_data_aadhar_enrolment_500000_1000000.csv
Saving api_data_aadhar_enrolment_0_500000.csv to api_data_aadhar_enrolment_0_500000.csv


In [3]:
!pip install prophet plotly gradio


In [4]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet

FILES = [
    "api_data_aadhar_enrolment_0_500000.csv",
    "api_data_aadhar_enrolment_500000_1000000.csv",
    "api_data_aadhar_enrolment_1000000_1006029.csv"
]

df = pd.concat([pd.read_csv(f) for f in FILES], ignore_index=True)

df["date"] = pd.to_datetime(df["date"], format="%d-%m-%Y", errors="coerce")
df = df.dropna(subset=["date"])

df["total_enrolments"] = (
    df["age_0_5"] + df["age_5_17"] + df["age_18_greater"]
)

df["year_month"] = df["date"].dt.to_period("M").astype(str)

national_month = (
    df.groupby("year_month")["total_enrolments"]
    .sum()
    .reset_index()
)

fig_growth = px.line(
    national_month,
    x="year_month",
    y="total_enrolments",
    title="National Aadhaar Enrolment Trend (Growth vs Saturation)",
    labels={"year_month": "Month", "total_enrolments": "Total Enrolments"}
)

fig_growth.show()



prophet_df = pd.DataFrame({
    "ds": pd.to_datetime(national_month["year_month"]),
    "y": national_month["total_enrolments"]
})

model = Prophet(yearly_seasonality=True)
model.fit(prophet_df)

future = model.make_future_dataframe(periods=6, freq="M")
forecast = model.predict(future)

fig_forecast = go.Figure()

fig_forecast.add_trace(go.Scatter(
    x=prophet_df["ds"],
    y=prophet_df["y"],
    name="Actual",
    mode="lines"
))

fig_forecast.add_trace(go.Scatter(
    x=forecast["ds"],
    y=forecast["yhat"],
    name="Forecast",
    mode="lines",
    line=dict(dash="dash")
))

fig_forecast.add_trace(go.Scatter(
    x=forecast["ds"],
    y=forecast["yhat_upper"],
    mode="lines",
    line=dict(width=0),
    showlegend=False
))

fig_forecast.add_trace(go.Scatter(
    x=forecast["ds"],
    y=forecast["yhat_lower"],
    mode="lines",
    fill="tonexty",
    name="Confidence Interval",
    line=dict(width=0)
))

fig_forecast.update_layout(
    title="Expected Aadhaar Enrolment Demand (Next 3–6 Months)",
    xaxis_title="Date",
    yaxis_title="Total Enrolments"
)

fig_forecast.show()



age_totals = df[["age_0_5", "age_5_17", "age_18_greater"]].sum().reset_index()
age_totals.columns = ["Age Group", "Total Enrolments"]

fig_age = px.bar(
    age_totals,
    x="Age Group",
    y="Total Enrolments",
    title="Aadhaar Enrolment by Age Group (Ecosystem Maturity Indicator)",
    labels={"Total Enrolments": "Cumulative Enrolments"}
)

fig_age.show()



INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 6.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [10]:
!pip install -U kaleido


In [12]:
fig_growth.write_html("national_growth_trend.html")
fig_forecast.write_html("national_enrolment_forecast.html")
fig_age.write_html("age_wise_enrolment.html")




In [13]:
from google.colab import files

files.download("national_growth_trend.html")
files.download("national_enrolment_forecast.html")
files.download("age_wise_enrolment.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>